<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, cv, Pool
from sklearn.preprocessing import LabelEncoder

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/auto-ru-download-2020-12-20csv/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'auto_ru_v2.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.head(5)

In [ ]:
train.info()

In [ ]:
test.head(5)

In [ ]:
test.info()

## Data Preprocessing

In [ ]:
# ... 

In [ ]:
y = train['price']

Удаляем ненужные колонки: технические id, с единственным значением на оба датасета (такие как Состояние). Также мы забыли сделать колонку vehicleConfiguration при скрейпинге датасета, поэтому удалим ее из тестового тоже. 

In [ ]:
train.drop(['Unnamed: 0', 'Unnamed: 0.1', 'color_hex', 'id','productionDate','priceCurrency','name','Состояние','Таможня'], axis=1, inplace=True)
test.drop(['car_url', 'image', 'parsing_unixtime','vehicleConfiguration','sell_id','priceCurrency', 'productionDate','name','Состояние','Таможня'], axis=1, inplace=True)

In [ ]:
train.rename(columns={"body_type":"bodyType"}, inplace=True)

In [ ]:
df_train = train.drop(["price"],axis=1)
df_test = test.copy()

## Processing

In [ ]:
cat_columns = ['bodyType', 'brand', 'color', 'fuelType','model_name', 'vehicleTransmission','vendor','ПТС','Привод','Руль']
num_values = ['engineDisplacement','enginePower', 'mileage','modelDate','numberOfDoors','productionDate','Владельцы','Владение']


In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

### Numeric Values

In [ ]:
data.engineDisplacement = np.log(1+data.engineDisplacement.apply(lambda v: float(v[0:-4]) if len(v[0:-4]) else 0))

In [ ]:
data.enginePower = np.log(1+data.enginePower.apply(lambda v: float(v[0:-4])))

In [ ]:
data.mileage = np.log(1+data.mileage)

In [ ]:
data["Владельцы"] = data["Владельцы"].map({"3 или более": 3, '1\xa0владелец': 1, '2\xa0владельца': 2, 0.:0., 1.:1., 2.:2., 3.:3. })

In [ ]:
data['Владение'].fillna(-1, inplace=True)

In [ ]:
import re
year_pattern = re.compile(r'([0123456789]*) \b(?:лет|год)')
month_pattern = re.compile(r'([0123456789]*) \bмесяц')

def parse_ownership_year_month(text):
    if isinstance(text, float) or isinstance(text, int):
        return 0, text
    year, month = (0,0)
    m = year_pattern.search(text)
    if m:        
        year = int(m.group(1))
    m = month_pattern.search(text)
    if m:        
        month = int(m.group(1))
    return year, month

parse_ownership_year_month('1 месяц и 2 года')

data['Владение'] = data['Владение'].apply(lambda v: (lambda y, m: y*12+m)(*parse_ownership_year_month(v)))

In [ ]:
data['Владение'] = np.log(2+data['Владение'])

### Categorical

In [ ]:
data.fuelType = data.fuelType.\
    map({'GASOLINE':'бензин','DIESEL':'дизель', 'HYBRID':'гибрид', 'ELECTRO':'электро', 'LPG':'газ',
        'бензин':'бензин','дизель':'дизель', 'гибрид':'гибрид', 'электро':'электро', 'газ':'газ'})

In [ ]:
data[data['ПТС'].isna()].mileage.value_counts()

ПТС отсутствует у новых авто с нулевым пробегом. Будем считать что для них есть оригинал.

In [ ]:
data['ПТС'].fillna("Оригинал", inplace=True)

### Unprocessed

In [ ]:
# Can be good to add features like clearance and acceleration to the model
#train.super_gen 
#description
#complectation_dict
#equipment_dict
#model_info
data.drop(["super_gen","description","equipment_dict","complectation_dict","model_info"],axis=1,inplace=True)

In [ ]:
#TODO 
# These tags can improve quality

import json
data.equipment_dict.fillna('{}', inplace=True)
        
data.equipment_dict.apply(lambda v: v if len(v.strip()) else '{}')\
    .apply(lambda v: v.replace("'","\""))\
    .apply(lambda v: v.replace("True","true"))\
    .apply(lambda v: json.loads(v).keys())

## Encoding

In [ ]:
for colum in cat_columns:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data

In [ ]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
def create_best_cat_boost_regressor():
    return CatBoostRegressor(iterations = 5000,
                          learning_rate = 0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          bagging_temperature=1,
                          one_hot_max_size=2,
                          random_strength=1,
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model = create_best_cat_boost_regressor()

model.fit(X_train, np.log(y_train),
         #cat_features=cat_columns,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         plot=True
         )

model.save_model('catboost_single_model_2.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Model 3 : Stacking

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

estimators = [
    ('cb',create_best_cat_boost_regressor()),
    ('rf',RandomForestRegressor(random_state=RANDOM_SEED))
]

reg = StackingRegressor(estimators=estimators)
reg.fit(X_train, np.log(y_train))

predict_test = np.exp(reg.predict(X_test))
predict_submission = np.exp(reg.predict(X_sub))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing